In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas


In [3]:
%load_ext rpy2.ipython
%R library(lme4)

Loading required package: lattice
Loading required package: Matrix


In [55]:
%R data <- read.csv('http://www.stat.wisc.edu/~ane/st572/data/floweringTime.txt', sep='\t')
%R data <- subset(data, !is.na(dtf))
%R data$logdtf <- log(data$dtf)
%R fit <- lmer(logdtf ~ (1|subspecies)+(1|inventoryID), data=data, REML=F)
%R print(summary(fit))


Linear mixed model fit by maximum likelihood ['lmerMod']
Formula: logdtf ~ (1 | subspecies) + (1 | inventoryID) 
   Data: data 

      AIC       BIC    logLik  deviance 
-118.0267 -106.4653   63.0134 -126.0267 

Random effects:
 Groups      Name        Variance Std.Dev.
 inventoryID (Intercept) 0.033431 0.1828  
 subspecies  (Intercept) 0.094382 0.3072  
 Residual                0.009644 0.0982  
Number of obs: 133, groups: inventoryID, 36; subspecies, 9

Fixed effects:
            Estimate Std. Error t value
(Intercept)   3.9756     0.1092   36.42


In [56]:
data = pandas.read_csv('http://www.stat.wisc.edu/~ane/st572/data/floweringTime.txt', delimiter='\t', index_col=0)
data = data.dropna()
data['logdtf'] = np.log(data['dtf'])
vcf = {"subspecies" : "0 + C(subspecies)", "inventoryID" : "0 + C(inventoryID)"}

model = sm.MixedLM.from_formula('logdtf ~ 1', groups="subspecies", vc_formula=vcf,  data=data)
result = model.fit(reml=False)
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
========================================================
Model:             MixedLM  Dependent Variable:  logdtf 
No. Observations:  133      Method:              ML     
No. Groups:        9        Scale:               0.0096 
Min. group size:   4        Likelihood:          63.0134
Max. group size:   32       Converged:           Yes    
Mean group size:   14.8                                 
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      3.976    0.109 36.415 0.000  3.762  4.190
inventoryID RE 0.033    0.114                           
subspecies RE  0.094    0.554                           
========================================================

"""

In [57]:
%R fit.noinventory <- update(fit, .~.- (1|inventoryID))
%R print(summary(fit.noinventory))


Linear mixed model fit by maximum likelihood ['lmerMod']
Formula: logdtf ~ (1 | subspecies) 
   Data: data 

     AIC      BIC   logLik deviance 
-23.3853 -14.7142  14.6926 -29.3853 

Random effects:
 Groups     Name        Variance Std.Dev.
 subspecies (Intercept) 0.10092  0.3177  
 Residual               0.03709  0.1926  
Number of obs: 133, groups: subspecies, 9

Fixed effects:
            Estimate Std. Error t value
(Intercept)    3.959      0.108   36.66


In [58]:
#data = pandas.read_csv('http://www.stat.wisc.edu/~ane/st572/data/floweringTime.txt', delimiter='\t', index_col=0)
#data = data.dropna()
#data['logdtf'] = np.log(data['dtf'])
vcf = {"subspecies" : "1 + C(subspecies)"}
model_noinventory = sm.MixedLM.from_formula('logdtf ~ 1', groups="subspecies",  vc_formula=vcf, data=data)
result_noinventory = model_noinventory.fit(reml=False)
result_noinventory.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:              MixedLM Dependent Variable: logdtf 
No. Observations:   133     Method:             ML     
No. Groups:         9       Scale:              0.0371 
Min. group size:    4       Likelihood:         14.6926
Max. group size:    32      Converged:          Yes    
Mean group size:    14.8                               
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     3.959    0.108 36.658 0.000  3.747  4.171
subspecies RE 0.101    0.270                           
=======================================================

"""

In [59]:
%R print(anova(fit, fit.noinventory))

Data: data
Models:
fit.noinventory: logdtf ~ (1 | subspecies)
fit: logdtf ~ (1 | subspecies) + (1 | inventoryID)
                Df      AIC      BIC logLik deviance  Chisq Chi Df Pr(>Chisq)
fit.noinventory  3  -23.385  -14.714 14.693  -29.385                         
fit              4 -118.027 -106.465 63.013 -126.027 96.641      1  < 2.2e-16
                   
fit.noinventory    
fit             ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [61]:
result.compare_lr_test(result_noinventory)


(96.641440352655081, 8.3090825662073875e-23, 1)

In [64]:
%R data <- read.table('http://www.stat.wisc.edu/~ane/st572/data/bilingual.txt', header=T)
%R fit = lmer(score ~ phase * time * language + (1|pair) + (1|child), data=data)
%R print(summary(fit))

Linear mixed model fit by REML ['lmerMod']
Formula: score ~ phase * time * language + (1 | pair) + (1 | child) 
   Data: data 

REML criterion at convergence: 1172.664 

Random effects:
 Groups   Name        Variance Std.Dev.
 child    (Intercept) 26.58    5.155   
 pair     (Intercept)  0.00    0.000   
 Residual             92.59    9.622   
Number of obs: 160, groups: child, 40; pair, 20

Fixed effects:
                                            Estimate Std. Error t value
(Intercept)                                   24.000      2.441   9.832
phasepre-switch                                6.500      3.043   2.136
timeend                                       34.250      3.043  11.256
languagemonolingual                           -1.000      3.452  -0.290
phasepre-switch:timeend                       -5.500      4.303  -1.278
phasepre-switch:languagemonolingual           -5.750      4.303  -1.336
timeend:languagemonolingual                  -17.500      4.303  -4.067
phasepre-switc

In [65]:
data = pandas.read_table('bilingual.txt', skipinitialspace=True)
vcf = {"pair" : "0 + C(pair)", "child" : "0 + C(child)"}
model = sm.MixedLM.from_formula('score ~  phase*time*language', groups='pair', vc_formula=vcf, data=data)
result = model.fit(reml=False)
print (result.summary())

                                Mixed Linear Model Regression Results
Model:                           MixedLM                Dependent Variable:                score    
No. Observations:                160                    Method:                            ML       
No. Groups:                      20                     Scale:                             87.9608  
Min. group size:                 8                      Likelihood:                        -600.4745
Max. group size:                 8                      Converged:                         Yes      
Mean group size:                 8.0                                                                
----------------------------------------------------------------------------------------------------
                                                         Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------------------------------------
Intercept            

/usr/local/lib/python2.7/dist-packages/statsmodels-0.7.0-py2.7-linux-x86_64.egg/statsmodels/regression/mixed_linear_model.py:1912: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [66]:
#%R fitno3way = lmer(score ~ (1|pair) + (1|child), data=data)
%R fitno3way <- update(fit, .~. - phase:time:language)
%R print(summary(fitno3way))


Linear mixed model fit by REML ['lmerMod']
Formula: score ~ phase + time + language + (1 | pair) + (1 | child) +      phase:time + phase:language + time:language 
   Data: data 

REML criterion at convergence: 1188.769 

Random effects:
 Groups   Name        Variance Std.Dev.
 child    (Intercept)  24.55    4.955  
 pair     (Intercept)   0.00    0.000  
 Residual             100.70   10.035  
Number of obs: 160, groups: child, 40; pair, 20

Fixed effects:
                                    Estimate Std. Error t value
(Intercept)                           26.531      2.373  11.178
phasepre-switch                        1.437      2.748   0.523
timeend                               29.187      2.748  10.621
languagemonolingual                   -6.063      3.163  -1.916
phasepre-switch:timeend                4.625      3.173   1.457
phasepre-switch:languagemonolingual    4.375      3.173   1.379
timeend:languagemonolingual           -7.375      3.173  -2.324

Correlation of Fixed Effec

In [67]:
modelno3way = sm.MixedLM.from_formula('score ~  phase*time*language - phase:language:time', groups='pair', vc_formula=vcf, data=data)
resultno3way = modelno3way.fit(reml=False)
print (resultno3way.summary())

                         Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           score    
No. Observations:             160               Method:                       ML       
No. Groups:                   20                Scale:                        102.3728 
Min. group size:              8                 Likelihood:                   -606.5522
Max. group size:              8                 Converged:                    No       
Mean group size:              8.0                                                      
---------------------------------------------------------------------------------------
                                            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
---------------------------------------------------------------------------------------
Intercept                                   26.531    2.288 11.594 0.000  22.046 31.016
phase[T.pre-switch]                          1.437    2.7

/usr/local/lib/python2.7/dist-packages/statsmodels-0.7.0-py2.7-linux-x86_64.egg/statsmodels/base/model.py:473: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/statsmodels-0.7.0-py2.7-linux-x86_64.egg/statsmodels/regression/mixed_linear_model.py:1894: ConvergenceWarning: Gradient optimization failed.
  warnings.warn(msg, ConvergenceWarning)


In [68]:
result.compare_lr_test(resultno3way)

(12.155308379806229, 0.00048948349354164796, 1)

In [69]:
%R print(anova(fit, fitno3way))


Data: data
Models:
fitno3way: score ~ phase + time + language + (1 | pair) + (1 | child) + 
fitno3way:     phase:time + phase:language + time:language
fit: score ~ phase * time * language + (1 | pair) + (1 | child)
          Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
fitno3way 10 1232.1 1262.8 -606.04   1212.1                             
fit       11 1223.0 1256.8 -600.47   1201.0 11.123      1  0.0008527 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
